# NSRR CCSHS Features Extraction

Unfortunately, for this dataset we do not have metric of signal quality and scoring reliability.

**WARNING:** 

- All EEG, EOG and EMG are referenced to Fpz.
- Sampling rate is 128 Hz with a highpass at 0.15 Hz.

https://sleepdata.org/datasets/ccshs/pages/montage-and-sampling-rate-information.md

In [ ]:
import os
import yasa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
from preprocessing import crop_hypno, extract_features

# Define paths
root_dir = '/Volumes/NSRR/ccshs/'
eeg_dir = root_dir + 'polysomnography/edfs/'
hypno_dir = root_dir + 'polysomnography/annotations-events-profusion/'
parent_dir = os.path.dirname(os.getcwd())
out_dir = parent_dir + '/output/features/'

# Keep training set of CCSHS only
df_subj = pd.read_csv(parent_dir + "/output/demo/demo_nsrr_all.csv")
df_subj = df_subj.query("dataset == 'CCSHS' and set == 'training'").set_index("subj")

print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

In [ ]:
df = []
include = ['C4', 'LOC', 'EMG1']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'ccshs-trec-' + sub + '.edf'
    hypno_file = hypno_dir + 'ccshs-trec-' + sub + '-profusion.xml'

    # LOAD EEG DATA
    try:
        # To deal with MNE loading error i_sub = 126
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw = read_raw_edf(eeg_file, preload=True, 
                           exclude=np.setdiff1d(raw.info['ch_names'], include), 
                           verbose=0)
    except:
        continue
        
    # Resample and low-pass filter 
    raw.resample(sf, npad="auto")
    
    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # We keep up to 15 minutes before / after sleep
    hypno, tmin, tmax = crop_hypno(hypno)
    # Crop EEG data
    raw.crop(tmin, tmax)
    # Check that hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    # stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA
    # Hypnogram must include all stages
    if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
        print("- Not all stages are present.")
        continue
    # Duration must be between 4 to 12 hours
    if not(4 < n_epochs / 120 < 12):
        print("- Recording too short/long.")
        continue

    # EXTRACT FEATURES
    features = extract_features(df_subj, sub, raw, include)
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    df.append(features)

df = pd.concat(df)

In [ ]:
# Add dataset
df['dataset'] = 'ccshs'

# Convert to category
df['dataset'] = df['dataset'].astype('category')
df['stage'] = df['stage'].astype('category')

In [ ]:
# Show %stage
df['stage'].value_counts(normalize=True, sort=True)

In [ ]:
# Median value of the EEG IQR per stage
df.groupby('stage')['eeg_iqr'].median()

In [ ]:
# Number of unique nights in dataset
df.index.get_level_values(0).nunique()

In [ ]:
# Export feature file
df.to_parquet(out_dir + "features_nsrr_ccshs.parquet")